In [1]:
import warnings
warnings.filterwarnings('ignore')

# Import crew librery
from crewai import Agent, Task, Crew

In [2]:
# Connecting to any LLM (NVIDIA)

from dotenv import load_dotenv
from crewai import Agent
from langchain_nvidia_ai_endpoints import ChatNVIDIA

import os

load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvapi_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

nvapi_key=os.environ.get("NVIDIA_API_KEY")

# Check for the available Nvidia LLM model
llm_nvidia = ChatNVIDIA(model="meta/llama3-8b-instruct", nvidia_api_key=nvapi_key)
llm_vision = ChatNVIDIA(model="microsoft/phi-3-vision-128k-instruct", nvidia_api_key=nvapi_key)

In [ ]:
import base64

with open("images/leccion4.jpg", "rb") as f:
  image_b64 = base64.b64encode(f.read()).decode()

assert len(image_b64) < 180_000, \
  "To upload larger images, use the assets API (see docs)"


Create Agents

In [3]:
mother = Agent(
    role="Madre",  # Mother -> Madre
    goal="Ayudar a su hijo a hacer la tarea de ciencias",  # Help her child with math homework -> Ayudar a su hijo a hacer la tarea de matemáticas
    backstory="Tu hijo te pide ayuda con su tarea de ciencias. "
             "Estás dispuesta a preguntarle la lección que le dejaron "
             "y a explicarle los conceptos que no entienda.",  # backstory -> translated in full
    llm=llm_vision,
    allow_delegation=False,
    verbose=True
)

planner = Agent(
    role="Planificador de Contenido",  # Content Planner -> Planificador de Contenido
    goal="Planificar contenido atractivo y factualmente preciso sobre {topic}",  # Plan engaging and factually accurate content on {topic} -> Planificar contenido atractivo y factualmente preciso sobre {topic}
    backstory="Estás trabajando en la planificación de un artículo de blog "
             "sobre el tema: {topic}. Recopilas información que ayuda a la "
             "audiencia a aprender algo y tomar decisiones informadas. "
             "Tu trabajo es la base para que el Redactor de Contenido escriba "
             "un artículo sobre este tema.",  # backstory -> translated in full
    llm=llm_nvidia,
    allow_delegation=False,
    verbose=True
)

writer = Agent(
    role="Redactor de Contenido",  # Content Writer -> Redactor de Contenido
    goal="Escribir un artículo de opinión perspicaz y factualmente preciso "
         "sobre el tema: {topic}",  # Write insightful and factually accurate opinion piece about the topic: {topic} -> Escribir un artículo de opinión perspicaz y factualmente preciso sobre el tema: {topic}
    backstory="Estás trabajando en la redacción de un nuevo artículo de opinión "
             "sobre el tema: {topic}. Basas tu redacción en el trabajo del "
             "Planificador de Contenido, quien te proporciona un esquema "
             "y contexto relevante sobre el tema. Sigues los objetivos "
             "principales y la dirección del esquema, tal como lo proporciona "
             "el Planificador de Contenido. También aportas perspectivas objetivas "
             "e imparciales y las respaldas con información proporcionada por el "
             "Planificador de Contenido. Reconoces en tu artículo de opinión "
             "cuándo tus declaraciones son opiniones en lugar de afirmaciones objetivas.",  # backstory -> translated in full
    llm=llm_nvidia,
    allow_delegation=False,
    verbose=True
)

editor = Agent(
    role="Editor",  # Maintains the role in English
    goal="Editar una publicación de blog dada para alinearla con el "
         "estilo de escritura de la organización.",  # Edit a given blog post to align with the writing style of the organization. -> Editar una publicación de blog dada para alinearla con el estilo de escritura de la organización.
    backstory="Eres un editor que recibe una publicación de blog "
             "del Redactor de Contenido. Tu objetivo es revisar la "
             "publicación para asegurarte de que sigue las mejores prácticas "
             "periodísticas, que proporciona puntos de vista equilibrados "
             "al proporcionar opiniones o afirmaciones, y que también evita "
             "temas o opiniones controvertidos importantes siempre que sea posible.",  # backstory -> translated in full
    llm=llm_nvidia,
    allow_delegation=False,
    verbose=True
)

Creating Task

In [4]:
ver = Task(
    description=(
        "1. Carga la imágen {image_b64} "  # Prioritize the latest trends, key players, and noteworthy news on {topic} -> Prioriza las últimas tendencias, actores clave y noticias destacadas sobre {topic}.
        "2. Esta imagen son los apuntes de una niña de 13 años en su clase de ciencias.\n"  # Identify the target audience, considering their interests and pain points -> Identifica el público objetivo, teniendo en cuenta sus intereses y puntos de dolor.
        "3. Identifica los conceptos que se muestran en la imagen.\n"  # Develop a detailed content outline including an introduction, key points, and a call to action -> Desarrolla un esquema de contenido detallado que incluya una introducción, puntos clave y una llamada a la acción.
        "4. Lanza 4 preguntas que se puedan responder con los datos de la imagen."  # Include SEO keywords and relevant data or sources -> Incluye palabras clave SEO y datos o fuentes relevantes.
    ),
    expected_output="La lista de preguntas ",  # A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources -> Un documento completo de planificación de contenido con un esquema, análisis de audiencia, palabras clave SEO y recursos.
    agent=mother,
)

plan = Task(
    description=(
        "1. Usa la lista de preguntas.\n"  # Prioritize the latest trends, key players, and noteworthy news on {topic} -> Prioriza las últimas tendencias, actores clave y noticias destacadas sobre {topic}.
        "2. Asegurate que el nivel de las preguntas es para un niño de 13 años.\n"  # Identify the target audience, considering their interests and pain points -> Identifica el público objetivo, teniendo en cuenta sus intereses y puntos de dolor.
        "3. Pon un título acorde con el tema.\n"  # Develop a detailed content outline including an introduction, key points, and a call to action -> Desarrolla un esquema de contenido detallado que incluya una introducción, puntos clave y una llamada a la acción.
    ),
    expected_output="Un documento completo de planificación de contenido ",  # A comprehensive content plan document with an outline, audience analysis, SEO keywords, and resources -> Un documento completo de planificación de contenido con un esquema, análisis de audiencia, palabras clave SEO y recursos.
    agent=planner,
)

write = Task(
    description=(
        "1. Revisa la publicación para detectar errores gramaticales y asegurar la alineación con la voz de la marca.\n"  # Proofread for grammatical errors and alignment with the brand's voice -> Revisa la publicación para detectar errores gramaticales y asegurar la alineación con la voz de la marca.
    ),
    expected_output="Una publicación de blog bien escrita en formato markdown, "
                   "lista para su publicación, cada sección debe tener de 2 a 3 párrafos.",  # A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs -> Una publicación de blog bien escrita en formato markdown, lista para su publicación, cada sección debe tener de 2 a 3 párrafos.
    agent=writer,
)

edit = Task(
    description=("Revisa la publicación de blog dada para detectar errores gramaticales "
                 "y asegurar la alineación con la voz de la marca."),  # Proofread the given blog post for grammatical errors and alignment with the brand's voice -> Revisa la publicación de blog dada para detectar errores gramaticales y asegurar la alineación con la voz de la marca.
    expected_output="Una publicación de blog bien escrita en formato markdown, "
                   "lista para su publicación, cada sección debe tener de 2 a 3 párrafos.",  # A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs -> Una publicación de blog bien escrita en formato markdown, lista para su publicación, cada sección debe tener de 2 a 3 párrafos.
    agent=editor
)

Creating the Crew

In [18]:
crew = Crew(
    agents=[mother, planner, writer, editor],
    tasks=[ver, plan, write, edit],
    verbose=2
)

2024-06-04 15:53:49,330 - 81488 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [20]:
result = crew.kickoff(inputs={"topic": image_b64})

from IPython.display import Markdown
Markdown(result)

 [DEBUG]: == Working Agent: Planificador de Contenido
 [INFO]: == Starting Task: 1. Prioriza las últimas tendencias, actores clave y noticias destacadas sobre Como fue el concierto de taylor swift en madrid en mayo 2024?.
2. Identifica el público objetivo, teniendo en cuenta sus intereses y puntos de dolor.
3. Desarrolla un esquema de contenido detallado que incluya una introducción, puntos clave y una llamada a la acción.
4. Incluye palabras clave SEO y datos o fuentes relevantes.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:


**Planificación de Contenido:** Como fue el concierto de taylor swift en madrid en mayo 2024?

**Análisis de Audiencia:**

* Público objetivo: Fanáticos de Taylor Swift, aficionados a la música, visitantes de Madrid y residentes en la ciudad.
* Intereses: Conciertos, música pop, Taylor Swift, Madrid.
* Puntos de dolor: Querer saber cómo fue el concierto, qué canciones se interpretaron, qué impresiones deja el 

**Cómo fue el concierto de Taylor Swift en Madrid en mayo 2024?**
==============================


**Introducción**
------------

El concierto de Taylor Swift en Madrid en mayo 2024 fue un producto de meses de anticipación, y finalmente, la noche del 15 de mayo, la artista norteamericana se presentó en el Wanda Metropolitano en forma impresionante. La relación con la audiencia fue instantánea, y desde el emblemático "Welcome to New York" hasta el criado "Love Story", la cantante interpretó una selección de canciones de sus últimos álbumes, incluyendo "Folklore", "Evermore" y "Reputation". La noche comenzó a las 9:00 PM, con Taylor Swift vestida de blanco y un diseño de ballet en la plataforma del escenario.

**Puntos clave**
----------------

La organización del concierto fue impecable, con una programación cuidadosa de escenario y iluminación que ganaron aplausos de la audiencia y de la crítica especializada. La sala estuvo atestada de turistas y residentes en la ciudad, que llegaron a Rey Juan Carlos desde las primeras horas de la mañana para asegurar sus entradas. La atmósfera en el estadio fue eléctrica, con momentos de éxtasis y sorpresa durante todo el concierto. La relación con la audiencia fue tan personal como siempre, con Taylor Swift compartiendo anécdotas yConference con la audiencia.

**Llamada a la acción**
---------------------

¿Tuviste la oportunidad de asistir al concierto de Taylor Swift en Madrid en mayo 2024? Comparte tus experiencias con nosotros en las redes sociales y utilízala como oportunidad de crear contenido para tus propios canales. También puedes seguirnos en nuestras redes sociales para mantenerse al tanto de los próximos conciertos y noticias de Taylor Swift.

**Conclusión**
--------------

El concierto de Taylor Swift en Madrid en mayo 2024 fue un evento emocionante y recordable. Los fans de la artista disfrutaron de una actuación sin fisuras, con una programación inteligente y una organización impecable.

**Notas**
---------

* Respaldos: [enlace]
* Palabras clave SEO utilizadas: Taylor Swift, concierto en Madrid, mayo 2024, música pop, conciertos en España, Taylor Swift en Madrid.